In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
trained_model = tf.keras.applications.InceptionV3(input_shape = (224, 224, 3),include_top=False, weights='imagenet')
trained_model.summary()


for layer in trained_model.layers:
  layer.trainable = False

last_layer = trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output



x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense  (1, activation='sigmoid')(x)

model = Model(trained_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001),loss = 'binary_crossentropy',metrics = ['accuracy'])


# base=r'drive/MyDrive/data/dataset'
base=r'dataset'
a=os.path.join(base)
b=os.path.join(a)
Train_Mirror=os.path.join(b, 'with_mask')
c=os.listdir(Train_Mirror)
Train_NoMirror=os.path.join(b, 'without_mask')
d=os.listdir( Train_NoMirror )


p=os.path.join(base)
q=os.path.join(p)
Test_Mirror=os.path.join(q, 'with_mask')
r=os.listdir( Test_Mirror )

Test_NoMirror=os.path.join(q, 'without_mask')
s=os.listdir( Test_NoMirror )

# Commented out IPython magic to ensure Python compatibility.
# %matplotlib inline



nrows = 4
ncols = 4

pic_index = 0
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_train_mirror_pix = [os.path.join(Train_Mirror, fname) for fname in c[ pic_index-8:pic_index]]

next_train_No_mirror_pix = [os.path.join(Train_NoMirror, fname) for fname in d[pic_index-8:pic_index]]

for i, img_path in enumerate(next_train_mirror_pix+next_train_No_mirror_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('On') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()



train_datagen = ImageDataGenerator( rescale = 1.0/255.,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest' )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(b,batch_size=64,class_mode='binary',target_size=(224, 224))

validation_generator =  test_datagen.flow_from_directory(q,batch_size=20,class_mode  = 'binary',target_size = (224, 224))

history = model.fit(train_generator,validation_data=validation_generator,steps_per_epoch=12,epochs=5,validation_steps=9,verbose=1)

acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(acc))

plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

#Save Model

model.save("mask_detector.h5")